In [20]:
import pandas as pd
from konlpy.tag import Okt, Kkma
from Token import UserTokenizers
import joblib
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [21]:
class TrainTransfromVect:
    def __init__(self):
        """
        info : 자연어를 벡터화하는 훈련과 데이터 변경을 수행하는 클래스
        process : 
        1) self.fit_run 수행
        2) self.transform_run 수행 
      ----------------------
      추가 개발 요구 사항
      1) []벡터 훈련 모델 저장 코드 추가
      2) [2023/05/12] 저장된 백터 훈련 모델 로드 코드 추가
      3) []백터 변환 함수 수정 필요(훈련 모델 로드 코드)  
        """
        self.ut_cls=UserTokenizers()

    def fitWP_TFIDF(self):
        """
        info : 훈련된 워드피스 토크나이저로 tfidf 벡터화 모델선언
        """        
        self.wp_tfidf= TfidfVectorizer(tokenizer=self.ut_cls.WordPieceTokenizer)
        return self.wp_tfidf   
    def fitBPE_TFIDF(self):
        """
        info : 훈련된 BPE 토크나이저로 tfidf 백터화 모델선언
        주의 : 토크나이저 훈련 모델에서 <unk>문제 해결 필요
        """
        self.bpe_tfidf = TfidfVectorizer(tokenizer= self.ut_cls.BPETokenizer)
        return self.bpe_tfidf
    def fitKKMA_TFIDF(self):
        """
        info : 훈련된 Konlpy.Kkma 토크나이저로 tfidf 백터화 모델선언
        사용주의: 메모리에러, 소요시간 주의
        """
        self.kkma_tfidf = TfidfVectorizer(tokenizer=self.ut_cls.konlpyNounsTokenizer)
        
        return self.kkma_tfidf
    def fitMP_TFIDF(self):
        """
        info : 훈련된 konlpy.Okt 토크나이저로 tfidf 벡터화 모델선언
        사용주의: 메모리에러, 소요시간 주의
        """
        self.mp_tfidf= TfidfVectorizer(tokenizer=self.ut_cls.konlyMorphsTokenizer)
        
        return self.mp_tfidf
    
    def fit_run(self, user_token_nm,data) :
        """
        info : tfidf 벡터화 수행 
        param user_token_nm : 토크나이저 선택 {'wp': '워드피스','bpe':'BPE','kkma':'꼬꼬마','mp':'Okt'}
        param data:백터 모델 훈련 데이터
        """ 
        if user_token_nm =='wp':
            self.vec_model= self.fitWP_TFIDF()
            # 모델 저장 코드 추가 필요
            
        elif user_token_nm =='bpe':
            self.vec_model= self.fitBPE_TFIDF()
            # 모델 저장 코드 추가 필요
        elif user_token_nm =='kkma':
            self.vec_model= self.fitKKMA_TFIDF()
            # 모델 저장 코드 추가 필요
        elif user_token_nm =='mp':
            self.vec_model= self.fitMP_TFIDF()
            # 모델 저장 코드 추가 필요
            
        else :
            # 모델 개발 할 때 혹은 인수인계자에게 디버깅하기 좋은 코드로 넘길 때 사용
            raise ValueError("user_token_nm이 올바르지 않습니다.['wp','bpe','kkma','mp'] ")
            # 서비스 운영을 할 때 기본설정하고, logging을 이용해서 로그를 남기거나 슬랙 알람/ 메일링 주는 모듈이 실행되게 함
            # self.vec_model = self.fitWP_TFIDF()
        self.vec_model.fit(data)
        ## 저장된 모델이 있는 경우 : 모델을 불러오는 모듈로 전환
    
    def transform_run(self, data, chunk_size):
        """
        info: tfidf 백터화 하여 np.array로 변환
        param data : 벡터로 변환하려는 데이터
        param chunk_size : np.array로 변환하는 단위, 데이터 수
        return vec_arr: np.array 
        """
        ## 데이터 수를 조정해서 데이터를 변환함
        data_len= len(data)
        for st_idx in tqdm(range(0, data_len, chunk_size)):
            tmp_data = data[st_idx:st_idx+chunk_size]
            
            if st_idx==0:
                vac_arr= self.vec_model.transform(tmp_data).toarray()
            else :
                tmp_data_arr= self.vec_model.transform(tmp_data).toarray()
                vac_arr= np.append(vac_arr,tmp_data_arr, axis=0)
        return vac_arr

In [22]:
ttfv=TrainTransfromVect()

In [5]:
train_df=pd.read_csv('./ratings_train.csv')

In [6]:
train_df.dropna(inplace=True)

In [32]:
ttfv.fit_run('wp', train_df['document'])

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [33]:
ttfv.vec_model

TfidfVectorizer(tokenizer=<bound method UserTokenizers.WordPieceTokenizer of <Token.UserTokenizers object at 0x7f2ded05bf70>>)

In [34]:
wp_tfidf_vec=ttfv.transform_run(train_df['document'][:200],2)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 858.52it/s]


In [35]:
wp_tfidf_vec.shape

(200, 9873)

In [6]:
from nlp_preprocessing import TrainTransfromVect

In [23]:
new_ttfv=TrainTransfromVect()

In [8]:
new_ttfv.fit_run('wp', train_df['document'])

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [1]:
new_wp_tfidf_vec = new_ttfv.transform_run(train_df['document'][:10000],chunk_size=50)

NameError: name 'new_ttfv' is not defined

In [24]:
new_ttfv.fit_run('bpe',train_df['document'])

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [25]:
ne_bpe_tfidf_vec = new_ttfv.transform_run(train_df['document'][:1000], chunk_size=100)

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 69.10it/s]


In [29]:
new_ttfv.fit_run('kkma',train_df['document'][:5000])

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [30]:
new_kkma_tfidf_vec=new_ttfv.transform_run_train_dfsform_run(train_df['document'][:100],chunk_size=10)

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10.25it/s]


In [31]:
new_ttfv.fit_run('mp',train_df['document'][:5000])

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [32]:
new_mp_tfidf_vec=new_ttfv.transform_run(train_df['document'][:100],chunk_size=10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 102.15it/s]


In [33]:
new_mp_tfidf_vec

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])